In [1]:
import pandas as pd

In [2]:
mapeamento = pd.concat([pd.read_excel('./dados/mapeamento/PRODLIST Ind 2022 x NCM 2022.xlsx',names=['prodlist','ncm']),pd.read_excel('./dados/mapeamento/PRODLIST Ind 2019 x NCM 2019.xlsx',names=['prodlist','ncm']),pd.read_excel('./dados/mapeamento/PRODLIST Ind 2016 x NCM 2016.xlsx',names=['prodlist','ncm'])], axis=0,ignore_index=True)


In [7]:
cnaes_selecionados = pd.read_excel('./dados/mapeamento/Cnaes selecionados.xlsx',names=['cnae','descricao'],dtype={'cnae':str,'descricao':str})

In [8]:
cnaes_selecionados.head(20)

,cnae,descricao
0,192,Fabricação de produtos derivados do petróleo
1,267,Fabricação de equipamentos e instrumentos ópti...
2,303,Fabricação de veículos ferroviários
3,161,Desdobramento de madeira
4,304,Fabricação de aeronaves
5,241,Produção de ferro-gusa e de ferroligas
6,285,Fabricação de máquinas e equipamentos de uso n...
7,310,Fabricação de móveis
8,162,"Fabricação de produtos de madeira, cortiça e m..."
9,103,"Fabricação de conservas de frutas, legumes e o..."


In [3]:
mapeamento.head()

,prodlist,ncm
0,0500.2010,2701.19.00
1,0500.2030,2701.11.00
2,0500.2030,2701.12.00
3,0500.2030,2701.19.00
4,0500.2030,2702.10.00


In [10]:
#fazer o merge de cnaes selecionadas com mapeamento de forma que o cnae com seja pareado com os 3 primeiros digitos da prodlist
cnaes_selecionados = cnaes_selecionados.merge(mapeamento, left_on='cnae', right_on=mapeamento['prodlist'].str[:3], how='inner')


In [12]:
cnaes_selecionados.drop_duplicates(subset=['cnae','descricao','ncm'], inplace=True)

In [38]:
exportacoes_mundo_2023 = pd.read_excel('./dados/comex/Exportacao Mundo 2024 NCM.xlsx',dtype={'Código NCM':str})


In [39]:
exportacoes_mundo_2023.shape

(8019, 3)

In [40]:
exportacoes_usa_2023 = pd.read_excel('./dados/comex/Exportacao EUA 2024 NCM.xlsx',dtype={'Código NCM':str})


In [41]:
exportacoes_usa_2023.head()

,Código NCM,Descrição NCM,2024 - Valor US$ FOB
0,27090010,Óleos brutos de petróleo,5830983860
1,72071200,Outros produtos semimanufaturados de ferro ou ...,2774470802
2,09011110,"Café não torrado, não descafeinado, em grão",1895591133
3,47032900,"Pastas químicas de madeira, à soda ou ao sulfa...",1551604956
4,72011000,"Ferro fundido bruto não ligado, que contenha, ...",1423332945


In [42]:
exportacoes_mundo_2023 = exportacoes_mundo_2023.rename(columns={'2024 - Valor US$ FOB': 'US$ FOB Mundo'}).merge(
    exportacoes_usa_2023.rename(columns={'2024 - Valor US$ FOB': 'US$ FOB USA'}),
    on=['Código NCM', 'Descrição NCM'],
    how='inner').fillna({'US$ FOB USA': 0})

In [43]:
exportacoes_mundo_2023['participacao USA'] = exportacoes_mundo_2023['US$ FOB USA'] / exportacoes_mundo_2023['US$ FOB Mundo']

In [37]:
exportacoes_mundo_2023.head()

,Código NCM,Descrição NCM,US$ FOB Mundo,US$ FOB USA,participacao USA
0,27090010,Óleos brutos de petróleo,44963878520,5.830984e+09,0.129682
1,12019000,"Soja, mesmo triturada, exceto para semeadura",42941894470,5.748195e+07,0.001339
2,26011100,"Minérios de ferro e seus concentrados, exceto ...",26574698749,5.410724e+06,0.000204
3,17011400,Outros açúcares de cana,15925320657,4.396613e+08,0.027608
4,09011110,"Café não torrado, não descafeinado, em grão",11331933183,1.895591e+09,0.167279


In [11]:
cnaes_selecionados.head()

,cnae,descricao,prodlist,ncm
0,192,Fabricação de produtos derivados do petróleo,1921.2010,2713.20.00
1,192,Fabricação de produtos derivados do petróleo,1921.2010,2713.90.00
2,192,Fabricação de produtos derivados do petróleo,1921.2020,2713.11.00
3,192,Fabricação de produtos derivados do petróleo,1921.2025,2711.14.00
4,192,Fabricação de produtos derivados do petróleo,1921.2027,2711.29.10


In [49]:
cnaes_selecionados['ncm'].str.replace('.','', regex=False)

0       27132000
1       27139000
2       27131100
3       27111400
4       27112910
          ...   
8806    84717010
8807    84717020
8808    84717030
8809    84717040
8810    84717090
Name: ncm, Length: 7324, dtype: object

In [50]:
#filtrar as exportações para o mundo e EUA apenas com as NCM's que estão nas cnaes selecionadas
exportacoes_mundo_2023 = exportacoes_mundo_2023[exportacoes_mundo_2023['Código NCM'].isin(cnaes_selecionados['ncm'].str.replace('.','', regex=False))]

In [51]:
#Criar um arquivo excel com 3 abas: uma com as cnaes selecionadas, outra com as exportações para o mundo e EUA e apenas com as NCM's sem repetições

with pd.ExcelWriter('./dados/comex/Exportacoes Mundo e USA 2024 NCM.xlsx') as writer:
    cnaes_selecionados.to_excel(writer, sheet_name='CNAEs Selecionadas', index=False)
    exportacoes_mundo_2023.to_excel(writer, sheet_name='Exportações Mundo e USA 2024', index=False)
    exportacoes_mundo_2023[['Código NCM', 'Descrição NCM']].drop_duplicates().to_excel(writer, sheet_name='NCMs', index=False)